**Task-1: Inner Workings of ResNet-152**


1. Baseline Setup

In [ ]:
from torchvision import models

# Load pre-trained ResNet-152
model = models.resnet152(pretrained=True)

In [ ]:
import torch.nn as nn

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # CIFAR-10 has 10 classes

In [ ]:
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze only the classification head
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(224),   # ResNet input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10('./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training function
def train_one_epoch(model, loader):
    model.train()
    correct, total, running_loss = 0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100 * correct / total

# Validation function
def validate(model, loader):
    model.eval()
    correct, total, running_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100 * correct / total

# Train for 5 epochs
epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


Epoch 1/5 | Train Loss: 0.6695, Train Acc: 77.97% | Val Loss: 0.5093, Val Acc: 82.75%
Epoch 2/5 | Train Loss: 0.5155, Train Acc: 82.19% | Val Loss: 0.5283, Val Acc: 81.92%
Epoch 3/5 | Train Loss: 0.4880, Train Acc: 83.20% | Val Loss: 0.4687, Val Acc: 83.98%
Epoch 4/5 | Train Loss: 0.4722, Train Acc: 83.56% | Val Loss: 0.4600, Val Acc: 84.43%
Epoch 5/5 | Train Loss: 0.4606, Train Acc: 84.17% | Val Loss: 0.4698, Val Acc: 84.16%


In [ ]:
# TASK-1
import torch
print(torch.cuda.is_available())

True


2. Residual Connections in Practice

a) Disable Skip Connection

In [ ]:
import torch.nn as nn
from torchvision import models

# Load pre-trained ResNet-152
model = models.resnet152(pretrained=True)

# Freeze backbone except classification head
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Replace final classification layer for CIFAR-10
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

# -------- Disable skip connections in a few blocks --------
# We will modify some Bottleneck blocks to remove the skip connection

# Access the first layer group (layer1)
for i, block in enumerate(model.layer1):
    if i < 2:  # modify first two residual blocks
        # Save original forward
        original_forward = block.forward

        # Redefine forward without skip connection
        def forward_no_skip(x):
            out = block.conv1(x)
            out = block.bn1(out)
            out = block.relu(out)
            out = block.conv2(out)
            out = block.bn2(out)
            out = block.relu(out)
            out = block.conv3(out)
            out = block.bn3(out)
            # Skip connection removed
            return block.relu(out)

        block.forward = forward_no_skip

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # This ensures all layers are on GPU

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
def train_one_epoch(model, loader):
    model.train()
    correct, total, running_loss = 0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return running_loss / len(loader), 100 * correct / total

In [ ]:
def validate(model, loader):
    model.eval()
    correct, total, running_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / len(loader), 100 * correct / total

In [ ]:
print(train_one_epoch)

<function train_one_epoch at 0x7fbdb003e2a0>


In [ ]:
epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

NameError: name 'train_loader' is not defined

In [ ]:
# Train for 5 epochs
epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


NameError: name 'train_one_epoch' is not defined